In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=df45e9009f2f879a4bf29693c4a6a4b0aa1e37a1eafd9000c434084b9d6cf9d6
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
import os
import time
import random
import multiprocessing

from pyspark import SparkContext

In [4]:
# Генерация 50000 случайных 32-битных целых чисел и запись их в файл
with open('random_integers_2.txt', 'w') as f:
    for _ in range(50000):
        f.write(f"{random.randint(0, 2**32 - 1)}\n")

In [5]:
# Функция подсчета количества простых множителей
def prime_factors_count(n):
    count = 0
    # Считаем количество появлений 2 в факторизации числа
    while n % 2 == 0:
        count += 1
        n //= 2
    # Считаем количество других простых множителей
    for i in range(3, int(n**0.5) + 1, 2):
        while n % i == 0:
            count += 1
            n //= i
    # Если n осталось простым числом больше 2
    if n > 2:
        count += 1
    return count

In [6]:
# Функция подсчета последовательным алгоритмом
def sequential_prime_factors_count(file_path):
    total_count = 0
    with open(file_path, 'r') as f:
        for line in f:
            number = int(line.strip())
            total_count += prime_factors_count(number)
    return total_count

In [7]:
# Функция подсчета с использованием multiprocessing
def worker(numbers):
    total_count = 0
    for number in numbers:
        total_count += prime_factors_count(number)
    return total_count

def multiprocessing_prime_factors_count(file_path):
    with open(file_path, 'r') as f:
        numbers = [int(line.strip()) for line in f]

    cpu_count = os.cpu_count()
    chunk_size = len(numbers) // cpu_count
    chunks = [numbers[i:i + chunk_size] for i in range(0, len(numbers), chunk_size)]

    with multiprocessing.Pool(cpu_count) as pool:
        results = pool.map(worker, chunks)

    return sum(results)

In [8]:
def spark_prime_factors_count(file_path):
    sc = SparkContext.getOrCreate()
    numbers = sc.textFile(file_path).map(lambda x: int(x.strip()))
    counts = numbers.map(prime_factors_count).reduce(lambda x, y: x + y)
    sc.stop()
    return counts

In [9]:
file_path = 'random_integers_2.txt'

In [10]:
# Sequential выполнение
start_time = time.time()
sequential_count = sequential_prime_factors_count(file_path)
end_time = time.time()
sequential_duration = end_time - start_time
print(f"Sequential результат: {sequential_count}, Продолжительность: {sequential_duration} секунд")

Sequential результат: 205984, Продолжительность: 78.1376519203186 секунд


In [11]:
# Multiprocessing выполнение
start_time = time.time()
multiprocessing_count = multiprocessing_prime_factors_count(file_path)
end_time = time.time()
multiprocessing_duration = end_time - start_time
print(f"Multiprocessing результат: {multiprocessing_count}, Продолжительность: {multiprocessing_duration} секунд")

Multiprocessing результат: 205984, Продолжительность: 77.44239616394043 секунд


In [12]:
# PySpark выполнение
start_time = time.time()
spark_count = spark_prime_factors_count(file_path)
end_time = time.time()
spark_duration = end_time - start_time
print(f"pySpart результат: {spark_count}, Продолжительность: {spark_duration} секунд")

pySpart результат: 205984, Продолжительность: 89.44616270065308 секунд
